<a href="https://colab.research.google.com/github/whitechocobread/Ai-project/blob/main/%EA%B8%B0%EB%A7%90%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
resolve_data = pd.read_excel('/content/drive/MyDrive/data.xlsx')

In [ ]:
resolve_data.sample(n=10)

,labels,Description
249,1,하이니켈 NCMA 배터리 개발
149,2,"SK하이닉스, 삼성전자 이어 제조업 2위. 금융권에선 국민은행 유일한 순위권. 정보..."
174,2,"삼성전자, 개인정보보호 투자 1위…삼성SDS·LGU+도 크게 늘려"
159,0,디지털 문해력
179,2,세계 전기차 배터리 시장에서 중국 업체들의 점유율 증가
176,2,"삼성전자, 개인정보보호 투자 1위…삼성SDS·LGU+도 크게 늘려"
116,1,‘2차전지 현재와 미래를 한눈에’…울산 K배터리 쇼 개최
90,-2,"SK하이닉스, 삼성전자와의 낸드 시장 점유율 격차를 역대 최저로 줄여"
60,2,삼성전자와 SK하이닉스의 4분기 실적이 상향 전망
14,-2,현재 매출 증대와 환경적 지속가능성의 균형 고민 중


In [ ]:
resolve_data.loc[(resolve_data['labels'] == "강한부정"), 'labels'] = -2 #강한부정 => -2
resolve_data.loc[(resolve_data['labels'] == "부정"), 'labels'] = -1  #부정 => -1
resolve_data.loc[(resolve_data['labels'] == "중립"), 'labels'] = 0  #중립 => 0
resolve_data.loc[(resolve_data['labels'] == "긍정"), 'labels'] = 1  #긍정 => 1
resolve_data.loc[(resolve_data['labels'] == "강한긍정"), 'labels'] = 2  #강한긍정 => 2

In [ ]:
data['labels'].unique()

TypeError: ignored

In [ ]:
data_list = []
for q, label in zip(resolve_data['Description '], resolve_data['labels'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[1])
print(data_list[2])
print(data_list[-2])
print(data_list[-1])

['삼성전자 폴더블폰 시장 점유율이 86%에서 72%로 하락', '-2']
['글로벌 폴더블폰 시장이 전년 대비 16%, 전분기 대비 215% 증가', '1']
['중국 업체들의 공격적인 제품 출시로 삼성전자 점유율 하락', '-1']
['회사 측의 관리 감독 부족', '-2']
['희망공모가액 25만7000~30만원', '0']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

191
64


In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git
# setup.py 파일에서 'onnxruntime==1.8.0'을 'onnxruntime' 또는 사용 가능한 다른 버전으로 변경하세요.
!pip install -e KoBERT/


fatal: destination path 'KoBERT' already exists and is not an empty directory.
Obtaining file:///content/KoBERT
  Preparing metadata (setup.py) ... done
  Attempting uninstall: kobert
    Found existing installation: kobert 0.2.3
    Uninstalling kobert-0.2.3:
      Successfully uninstalled kobert-0.2.3
  Running setup.py develop for kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[0]

(array([   2, 3010, 5712, 3185,  703, 5208, 7640, 7342,  703, 3084, 5655,
        6255,  517,   55, 1293,  517,   54, 5152, 6730, 6441, 5859, 2638,
         518, 1907, 6115, 6629, 5384, 1698, 7361, 6003,  518,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(32, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 -2)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

RuntimeError: ignored

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# train_dataloader 및 test_dataloader를 사용하여 레이블 범위 확인
max_label = -float('inf')
min_label = float('inf')

for batch in train_dataloader:
    labels = batch[-1]  # 레이블은 배치의 마지막 요소라고 가정
    max_label = max(max_label, labels.max().item())
    min_label = min(min_label, labels.min().item())

for batch in test_dataloader:
    labels = batch[-1]  # 레이블은 배치의 마지막 요소라고 가정
    max_label = max(max_label, labels.max().item())
    min_label = min(min_label, labels.min().item())

print("레이블 최소값:", min_label)
print("레이블 최대값:", max_label)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


레이블 최소값: -2
레이블 최대값: 2


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-21-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/3 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
import numpy as np

# 예제 행렬 A를 정의합니다. 실제 행렬을 사용해야 한다면, 이 부분을 실제 값으로 대체하세요.
A = np.array([[2, 4], [5, -10]])  # 실제 행렬 값으로 채우세요.

# L1 노름 계산
l1_norm = np.linalg.norm(A, 1)

# 무한대 노름 계산
inf_norm = np.linalg.norm(A, np.inf)

# 프로베니우스 노름 계산
fro_norm = np.linalg.norm(A, 'fro')

print("L1 노름:", l1_norm)
print("무한대 노름:", inf_norm)
print("프로베니우스 노름:", fro_norm)


L1 노름: 14.0
무한대 노름: 15.0
프로베니우스 노름: 12.041594578792296
